<a href="https://colab.research.google.com/github/bwnyasse/learning-box/blob/main/google-cloud/colab/langchain_bigquery/langchain_bigquery_chat_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of required libraries.

In [3]:
!pip install langchain openai chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery

# Importing libraries.





In [4]:
from google.colab import userdata
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor


# BigQuery Credentials - Service account

In [5]:
# Set Up BigQuery Credentials
service_account_file = "/content/sa-keys/sa-langchain-test-over-bigquery.json"  # Adjust the path as needed

# SQL Achemy Configuration

In [6]:
# Configuration
project = "learning-box-369917"
dataset = "langchain_test"
table = "churn_table"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'

# Get an OpenAI key

In [8]:
os.environ["OPENAI_API_KEY"] = userdata.get('openAiApiKey')

# Initialize LangChain

Set up the LangChain with the specified configurations to prepare for executing queries.


In [9]:
db = SQLDatabase.from_uri(sqlalchemy_url)
llm = OpenAI(temperature=0, model="text-davinci-003")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=1000,
)

# Execute and Display Queries

Execute sample queries using the LangChain agent and display the responses.


In [10]:
agent_executor.run("How many male users churned?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: churn_table
Thought: I should query the schema of the churn_table
Action: sql_db_schema
Action Input: churn_table
Observation: 
CREATE TABLE `churn_table` (
	`customerID` STRING, 
	`gender` STRING, 
	`SeniorCitizen` INT64, 
	`Partner` BOOL, 
	`Dependents` BOOL, 
	`tenure` INT64, 
	`PhoneService` BOOL, 
	`MultipleLines` STRING, 
	`InternetService` STRING, 
	`OnlineSecurity` STRING, 
	`OnlineBackup` STRING, 
	`DeviceProtection` STRING, 
	`TechSupport` STRING, 
	`StreamingTV` STRING, 
	`StreamingMovies` STRING, 
	`Contract` STRING, 
	`PaperlessBilling` BOOL, 
	`PaymentMethod` STRING, 
	`MonthlyCharges` FLOAT64, 
	`TotalCharges` STRING, 
	`Churn` BOOL
)

/*
3 rows from churn_table table:
customerID	gender	SeniorCitizen	Partner	Dependents	tenure	PhoneService	MultipleLines	InternetService	OnlineSecurity	OnlineBackup	DeviceProtection	TechSupport	StreamingTV	StreamingMovies	Contract	PaperlessBilling	

'930 male users churned.'